### This jupyterntoebook is to join the domain data gotten from part 1 to crime, school merged data and suburb economics data

In [1]:
from pyspark.sql import SparkSession, functions as F
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType
import json
import re
import pandas as pd
import numpy as np
import requests
from pyspark.sql.functions import lower
from pyspark.sql.functions import col


# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("MAST30034 Domain EDA 2")
    .config("spark.sql.repl.eagerEval.enabled", True)
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.driver.port", "6066")
    .getOrCreate()
)

your 131072x1 screen size is bogus. expect trouble
24/10/07 08:44:53 WARN Utils: Your hostname, DESKTOP-VD1EQUL resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/10/07 08:44:53 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/07 08:44:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 49914)
Traceback (most recent call last):
  File "/usr/lib/python3.10/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib/python3.10/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib/python3.10/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib/python3.10/socketserver.py", line 747, in __init__
    self.handle()
  File "/home/chin_chee_henn/.local/lib/python3.10/site-packages/pyspark/accumulators.py", line 295, in handle
    poll(accum_updates)
  File "/home/chin_chee_henn/.local/lib/python3.10/site-packages/pyspark/accumulators.py", line 267, in poll
    if self.rfile in r and func():
  File "/home/chin_chee_henn/.local/lib/python3.10/site-packages/pyspark/accumulators.py", lin

In [2]:
# Get all the good stuff
domain_clean = spark.read.parquet("../../data/curated/domain_cleaned_pyspark", header=True)
full_merged = spark.read.csv("../../data/raw/merged_crime_schools_distances_land.csv", header=True)
final_economics = spark.read.csv("../../data/raw/final.csv", header=True)
merged_suburb = spark.read.csv("../../data/raw/merged_as_suburbs.csv", header=True)

**Domain Cleaned Data**


In [3]:
domain_clean.limit(2)

24/10/07 08:45:05 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


url,name,latitude,longitude,bedrooms,bathrooms,type,car_parks,cost_text,suburb,postcode,rent_pw,school_name,school_walking,school_driving,hospital_name,hospital_walking,hospital_driving,doctors_name,doctors_walking,doctors_driving,bank_name,bank_walking,bank_driving,restaurant_name,restaurant_walking,restaurant_driving,PTV_name,PTV_walking,PTV_driving,rate_type,cost_per_week
https://www.domai...,8/94 Kerferd Stre...,-37.73580320000001,144.9000261,2,1,Apartment / Unit ...,1,$475 per week,Essendon North V...,3041,475,Parkville College,32.84,32.84,Dorset Rehabilita...,13.37,20.32,Professional Foot...,7.72,8.2,Commonwealth Bank,2.93,13.41,Jan Cheong Restau...,6.28,12.65,Essendon Railway ...,24.31,39.23,weekly,475.0
https://www.domai...,402/9 Shuter Stre...,-37.7677644,144.9205115,2,1,Apartment / Unit ...,1,$520 pw / $2260 pcm,Moonee Ponds VIC...,3039,520,Parkville College,32.84,32.84,Unknown,17.6,17.6,Professional Foot...,7.72,8.2,ANZ,4.16,12.71,I Dream of Sushi,3.37,10.4,Essendon Railway ...,24.31,39.23,weekly,2260.0


In [4]:
domain_clean.count()

9041

In [5]:
domain_clean.dtypes

[('url', 'string'),
 ('name', 'string'),
 ('latitude', 'string'),
 ('longitude', 'string'),
 ('bedrooms', 'bigint'),
 ('bathrooms', 'bigint'),
 ('type', 'string'),
 ('car_parks', 'bigint'),
 ('cost_text', 'string'),
 ('suburb', 'string'),
 ('postcode', 'string'),
 ('rent_pw', 'bigint'),
 ('school_name', 'string'),
 ('school_walking', 'double'),
 ('school_driving', 'double'),
 ('hospital_name', 'string'),
 ('hospital_walking', 'double'),
 ('hospital_driving', 'double'),
 ('doctors_name', 'string'),
 ('doctors_walking', 'double'),
 ('doctors_driving', 'double'),
 ('bank_name', 'string'),
 ('bank_walking', 'double'),
 ('bank_driving', 'double'),
 ('restaurant_name', 'string'),
 ('restaurant_walking', 'double'),
 ('restaurant_driving', 'double'),
 ('PTV_name', 'string'),
 ('PTV_walking', 'double'),
 ('PTV_driving', 'double'),
 ('rate_type', 'string'),
 ('cost_per_week', 'double')]

In [6]:
# Count unique items in the 'letter' column
unique_count = domain_clean.select('suburb').distinct().count()
print(f"Number of unique items in 'suburb' column: {unique_count}")

Number of unique items in 'suburb' column: 692


In [7]:
pattern = r'.* VIC \d{4}$'

# Filter rows that match the pattern
domain_clean = domain_clean.filter(F.col('suburb').rlike(pattern))

In [8]:
domain_clean = domain_clean.withColumn('location', F.split(F.col('suburb'), ' VIC ')[0])
domain_clean = domain_clean.drop("suburb")
domain_clean = domain_clean.withColumnRenamed("location", "suburb")

In [9]:
domain_clean.limit(1)

url,name,latitude,longitude,bedrooms,bathrooms,type,car_parks,cost_text,postcode,rent_pw,school_name,school_walking,school_driving,hospital_name,hospital_walking,hospital_driving,doctors_name,doctors_walking,doctors_driving,bank_name,bank_walking,bank_driving,restaurant_name,restaurant_walking,restaurant_driving,PTV_name,PTV_walking,PTV_driving,rate_type,cost_per_week,suburb
https://www.domai...,8/94 Kerferd Stre...,-37.73580320000001,144.9000261,2,1,Apartment / Unit ...,1,$475 per week,3041,475,Parkville College,32.84,32.84,Dorset Rehabilita...,13.37,20.32,Professional Foot...,7.72,8.2,Commonwealth Bank,2.93,13.41,Jan Cheong Restau...,6.28,12.65,Essendon Railway ...,24.31,39.23,weekly,475.0,Essendon North


In [10]:
domain_clean.count()

9039

In [11]:
from pyspark.sql.functions import trim

# Removing all empty spaces in suburb
domain_clean = domain_clean.withColumn('suburb', trim(domain_clean['suburb']))


In [12]:
domain_suburb = domain_clean.select("suburb")

In [13]:
domain_suburb = domain_suburb.dropDuplicates(['suburb'])

**Big Merged Data by CInque**

In [14]:
full_merged = full_merged.drop("_c0")

In [15]:
full_merged.limit(1)

suburbs,year,quarter,a20_assault_and_related_offences,a50_robbery,"a70_stalking,_harassment_and_threatening_behaviour",a80_dangerous_and_negligent_acts_endangering_people,other_crimes_against_the_person,b10_arson,b20_property_damage,b30_burglary/break_and_enter,b40_theft,b50_deception,c10_drug_dealing_and_trafficking,c20_cultivate_or_manufacture_drugs,c30_drug_use_and_possession,d10_weapons_and_explosives_offences,d20_disorderly_and_offensive_conduct,d30_public_nuisance_offences,e10_justice_procedures,e20_breaches_of_orders,f20_transport_regulation_offences,f90_miscellaneous_offences,c90_other_drug_offences,f30_other_government_regulatory_offences,d40_public_security_offences,f10_regulatory_driving_offences,b60_bribery,total_crimes,best_school_number_vce_subjects,best_school_satisfactory_complete_vce_percent,best_school_median_study_score,best_school_study_score_over_40_percent,best_school_percentage_applying_to_victorian_uni,best_school_school_name,avg_school_number_vce_subjects,avg_school_satisfactory_complete_vce_percent,avg_school_median_study_score,avg_school_study_score_over_40_percent,avg_school_percentage_applying_to_victorian_uni,zoned_school_number_vce_subjects,zoned_school_satisfactory_complete_vce_percent,zoned_school_median_study_score,zoned_school_study_score_over_40_percent,zoned_school_percentage_applying_to_victorian_uni,crow_distance_to_cbd,distance_to_cbd,duration_to_cbd,route_to_cbd,total_area,developed_area,built_percentage,urban_percentage,disturbed_percentage,water_percentage,developed_percentage,nature_percentage
Albert Park-Middl...,2000,1,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,57.0,99.0,31.0,6.3,83.0,albert park college,57.0,99.0,31.0,6.3,83.0,50.954000614146445,98.48018046863834,30.9377258624176,6.76454713423704,78.99358325602229,4.53585711395057,6702.5,811.4,"[[144.967419, -37...",7043125.0,4691875.0,0.0322122637323631,0.595793770520898,0.0381577779749756,0.0473866359038069,0.6661638122282367,0.2864495518679563


In [16]:
full_merged.dtypes

[('suburbs', 'string'),
 ('year', 'string'),
 ('quarter', 'string'),
 ('a20_assault_and_related_offences', 'string'),
 ('a50_robbery', 'string'),
 ('a70_stalking,_harassment_and_threatening_behaviour', 'string'),
 ('a80_dangerous_and_negligent_acts_endangering_people', 'string'),
 ('other_crimes_against_the_person', 'string'),
 ('b10_arson', 'string'),
 ('b20_property_damage', 'string'),
 ('b30_burglary/break_and_enter', 'string'),
 ('b40_theft', 'string'),
 ('b50_deception', 'string'),
 ('c10_drug_dealing_and_trafficking', 'string'),
 ('c20_cultivate_or_manufacture_drugs', 'string'),
 ('c30_drug_use_and_possession', 'string'),
 ('d10_weapons_and_explosives_offences', 'string'),
 ('d20_disorderly_and_offensive_conduct', 'string'),
 ('d30_public_nuisance_offences', 'string'),
 ('e10_justice_procedures', 'string'),
 ('e20_breaches_of_orders', 'string'),
 ('f20_transport_regulation_offences', 'string'),
 ('f90_miscellaneous_offences', 'string'),
 ('c90_other_drug_offences', 'string'),
 ('

In [17]:
# Count unique items in the 'letter' column
unique_count = full_merged.select('suburbs').distinct().count()
print(f"Number of unique items in 'suburb' column: {unique_count}")

Number of unique items in 'suburb' column: 144


In [18]:
full_merged_suburbs = full_merged.select("suburbs")

In [19]:
full_merged_suburbs = full_merged_suburbs.dropDuplicates(['suburbs'])

In [20]:
from pyspark.sql.functions import split, explode

def splitting_suburb(df):
    # Split suburb names by the delimiter "-"
    df_split = df.withColumn("suburb_split", F.split(F.col("suburbs"), "-"))

    # Explode the array into separate rows, and trim any leading/trailing spaces
    df_exploded = df_split.withColumn("suburb", explode(df_split["suburb_split"]))

    df_result = df_exploded.drop("suburb_split", "suburbs")

    return df_result

In [21]:
full_merged_suburbs = splitting_suburb(full_merged_suburbs)
full_merged_suburbs.count()

213

In [22]:
full_merged_suburbs.head(1)

[Row(suburb='Bendigo')]

In [23]:
# Subtract df2 from df1
df_diff = domain_suburb.select('suburb').subtract(full_merged_suburbs.select('suburb'))

# Collect the result as a list
items_not_in_df2 = df_diff.rdd.flatMap(lambda x: x).collect()

# Show the list
print(items_not_in_df2)

['Brunswick West', 'Port Fairy', 'Ecklin South', 'Ashwood', 'South Kingsville', 'Boneo', 'Lucas', 'Daylesford', 'Brookfield', 'Doveton', 'Lynbrook', 'Fraser Rise', 'Highton', 'Blackburn North', 'Sandringham', 'St Kilda East', 'Sunshine North', 'Clayton South', 'Dallas', 'Rye', 'Leneva', 'Orbost', 'Lake Gardens', 'Rosanna', 'Mirboo North', 'Altona Meadows', 'Kalkallo', 'Caroline Springs', 'Ballarat North', 'Beechworth', 'Mount Duneed', 'Greenvale', 'Broadford', 'Wandana Heights', 'Narre Warren South', 'Werribee South', 'Macleod', 'Hallam', 'Oakleigh East', 'Sunset Strip', 'North Wonthaggi', 'Mickleham', 'Ripponlea', 'Sunshine West', 'Trentham', 'Myrniong', 'Queenscliff', 'Truganina', 'Drysdale', 'Sorrento', 'McKinnon', 'Blackburn South', 'Nichols Point', 'Gisborne', 'Huntly', 'Eganstown', 'Mount Evelyn', 'Bonshaw', 'Clifton Springs', 'Darley', 'Cobblebank', 'Laverton', 'Thornhill Park', 'Bonbeach', 'Ballarat East', 'St Albans Park', 'Norlane', 'Warrandyte', 'Mount Pleasant', 'Dandenong 

In [24]:
# Assuming 'suburbs' is the column name in df1
df_filtered = domain_clean.filter(domain_clean['suburb'].isin(items_not_in_df2))

# Show the result
df_filtered.count()

4115

In [25]:
from pyspark.sql import functions as F

# Filter rows where the year is 2024
full_merged_2024 = full_merged.filter((F.col("year") == 2024) & (F.col("quarter") == 2))
full_merged_no_null = full_merged.dropna()

In [26]:
from pyspark.sql.functions import split, explode

def splitting_suburb(df):
    # Split suburb names by the delimiter "-"
    df_split = df.withColumn("suburb_split", F.split(F.col("suburbs"), "-"))

    # Explode the array into separate rows, and trim any leading/trailing spaces
    df_exploded = df_split.withColumn("suburb", explode(df_split["suburb_split"]))

    df_result = df_exploded.drop("suburb_split", "suburbs")

    return df_result

In [27]:
full_merged_2024 = splitting_suburb(full_merged_2024)
full_merged_no_null = splitting_suburb(full_merged_no_null)

In [28]:
from pyspark.sql import Window
from pyspark.sql.functions import col, row_number

# Define a window partitioned by 'suburb', ordered by 'year' and 'quarter' descending
window_spec = Window.partitionBy('suburb').orderBy(col('year').desc(), col('quarter').desc())

# Add a row number for each partition (suburb) based on the order of year and quarter
df_with_row_num = full_merged_no_null.withColumn('row_num', row_number().over(window_spec))

# Filter to get the latest year and quarter for each suburb
full_merged_no_null = df_with_row_num.filter(col('row_num') == 1).drop('row_num')

In [29]:
full_merged_2024 = full_merged_2024.drop("total_area", "developed_area",
                                         "built_percentage", "urban_percentage",
                                         "disturbed_percentage", "water_percentage",
                                         "developed_percentage", "nature_percentage")

full_merged_no_null = full_merged_no_null.select("total_area", "developed_area",
                                        "built_percentage", "urban_percentage",
                                        "disturbed_percentage", "water_percentage",
                                        "developed_percentage", "nature_percentage",
                                        "suburb")


In [30]:
full_merged_2024 = full_merged_2024.join(full_merged_no_null,
                                         on="suburb",
                                         how="left")

In [31]:
full_merged_2024.limit(3)

suburb,year,quarter,a20_assault_and_related_offences,a50_robbery,"a70_stalking,_harassment_and_threatening_behaviour",a80_dangerous_and_negligent_acts_endangering_people,other_crimes_against_the_person,b10_arson,b20_property_damage,b30_burglary/break_and_enter,b40_theft,b50_deception,c10_drug_dealing_and_trafficking,c20_cultivate_or_manufacture_drugs,c30_drug_use_and_possession,d10_weapons_and_explosives_offences,d20_disorderly_and_offensive_conduct,d30_public_nuisance_offences,e10_justice_procedures,e20_breaches_of_orders,f20_transport_regulation_offences,f90_miscellaneous_offences,c90_other_drug_offences,f30_other_government_regulatory_offences,d40_public_security_offences,f10_regulatory_driving_offences,b60_bribery,total_crimes,best_school_number_vce_subjects,best_school_satisfactory_complete_vce_percent,best_school_median_study_score,best_school_study_score_over_40_percent,best_school_percentage_applying_to_victorian_uni,best_school_school_name,avg_school_number_vce_subjects,avg_school_satisfactory_complete_vce_percent,avg_school_median_study_score,avg_school_study_score_over_40_percent,avg_school_percentage_applying_to_victorian_uni,zoned_school_number_vce_subjects,zoned_school_satisfactory_complete_vce_percent,zoned_school_median_study_score,zoned_school_study_score_over_40_percent,zoned_school_percentage_applying_to_victorian_uni,crow_distance_to_cbd,distance_to_cbd,duration_to_cbd,route_to_cbd,total_area,developed_area,built_percentage,urban_percentage,disturbed_percentage,water_percentage,developed_percentage,nature_percentage
Albert Park,2024,2,193.72356466317277,17.57871728400593,56.50364392434726,22.259133314148805,98.11578342608074,3.9880208789769953,179.4308215269279,154.5100714296851,918.7136511150776,251.20118021952703,58.174847714297336,7.836771668748077,112.15705600660787,60.965884781128445,50.584491765696015,25.559150283130748,60.50053691440294,288.7700388420077,1.282853894517082,1.9363668638025688,0.1616815356166832,3.500012096453719,18.916739668308946,0.0083470330350795,0.0,2586.379366849703,57.0,99.0,31.0,6.3,83.0,albert park college,57.0,99.0,31.0,6.3,83.0,50.954000614146445,98.48018046863834,30.9377258624176,6.76454713423704,78.99358325602229,4.53585711395057,6702.5,811.4,"[[144.967419, -37...",7043125.0,4928125.0,0.0247581861744609,0.6343952435886059,0.0405537314757298,0.0468542017925281,0.6997071612387967,0.2534386369686751
Middle Park,2024,2,193.72356466317277,17.57871728400593,56.50364392434726,22.259133314148805,98.11578342608074,3.9880208789769953,179.4308215269279,154.5100714296851,918.7136511150776,251.20118021952703,58.174847714297336,7.836771668748077,112.15705600660787,60.965884781128445,50.584491765696015,25.559150283130748,60.50053691440294,288.7700388420077,1.282853894517082,1.9363668638025688,0.1616815356166832,3.500012096453719,18.916739668308946,0.0083470330350795,0.0,2586.379366849703,57.0,99.0,31.0,6.3,83.0,albert park college,57.0,99.0,31.0,6.3,83.0,50.954000614146445,98.48018046863834,30.9377258624176,6.76454713423704,78.99358325602229,4.53585711395057,6702.5,811.4,"[[144.967419, -37...",7043125.0,4928125.0,0.0247581861744609,0.6343952435886059,0.0405537314757298,0.0468542017925281,0.6997071612387967,0.2534386369686751
West St Kilda,2024,2,193.72356466317277,17.57871728400593,56.50364392434726,22.259133314148805,98.11578342608074,3.9880208789769953,179.4308215269279,154.5100714296851,918.7136511150776,251.20118021952703,58.174847714297336,7.836771668748077,112.15705600660787,60.965884781128445,50.584491765696015,25.559150283130748,60.50053691440294,288.7700388420077,1.282853894517082,1.9363668638025688,0.1616815356166832,3.500012096453719,18.916739668308946,0.0083470330350795,0.0,2586.379366849703,57.0,99.0,31.0,6.3,83.0,albert park college,57.0,99.0,31.0,6.3,83.0,50.954000614146445,98.48018046863834,30.9377258624176,6.76454713423704,78.99358325602229,4.53585711395057,6702.5,811.4,"[[144.967419, -37...",7043125.0,4928125.0,0.0247581861744609,0.6343952435886059,0.0405537314757298,0.0

In [32]:
full_merged_2024 = full_merged_2024.dropna()

In [33]:
# Calculate the sum of unique values in the 'value' column
unique_count = full_merged_2024.select("suburb").distinct().count()

# Display the result
print(f"Number of unique items in 'category' column: {unique_count}")

Number of unique items in 'category' column: 178


In [34]:
# Merge DataFrames on 'Name' column
partial_joined = domain_clean.join(full_merged_2024, on="suburb", how="left")

In [35]:
partial_joined.limit(5)

suburb,url,name,latitude,longitude,bedrooms,bathrooms,type,car_parks,cost_text,postcode,rent_pw,school_name,school_walking,school_driving,hospital_name,hospital_walking,hospital_driving,doctors_name,doctors_walking,doctors_driving,bank_name,bank_walking,bank_driving,restaurant_name,restaurant_walking,restaurant_driving,PTV_name,PTV_walking,PTV_driving,rate_type,cost_per_week,year,quarter,a20_assault_and_related_offences,a50_robbery,"a70_stalking,_harassment_and_threatening_behaviour",a80_dangerous_and_negligent_acts_endangering_people,other_crimes_against_the_person,b10_arson,b20_property_damage,b30_burglary/break_and_enter,b40_theft,b50_deception,c10_drug_dealing_and_trafficking,c20_cultivate_or_manufacture_drugs,c30_drug_use_and_possession,d10_weapons_and_explosives_offences,d20_disorderly_and_offensive_conduct,d30_public_nuisance_offences,e10_justice_procedures,e20_breaches_of_orders,f20_transport_regulation_offences,f90_miscellaneous_offences,c90_other_drug_offences,f30_other_government_regulatory_offences,d40_public_security_offences,f10_regulatory_driving_offences,b60_bribery,total_crimes,best_school_number_vce_subjects,best_school_satisfactory_complete_vce_percent,best_school_median_study_score,best_school_study_score_over_40_percent,best_school_percentage_applying_to_victorian_uni,best_school_school_name,avg_school_number_vce_subjects,avg_school_satisfactory_complete_vce_percent,avg_school_median_study_score,avg_school_study_score_over_40_percent,avg_school_percentage_applying_to_victorian_uni,zoned_school_number_vce_subjects,zoned_school_satisfactory_complete_vce_percent,zoned_school_median_study_score,zoned_school_study_score_over_40_percent,zoned_school_percentage_applying_to_victorian_uni,crow_distance_to_cbd,distance_to_cbd,duration_to_cbd,route_to_cbd,total_area,developed_area,built_percentage,urban_percentage,disturbed_percentage,water_percentage,developed_percentage,nature_percentage
Aintree,https://www.domai...,"11 Cascade Drive,...",-37.7196462,144.6775788,4,2,House,2,$490,3336,490,Parkville College,32.84,32.84,Keilor Downs Surgery,15.59,15.68,Deer Park Doctors,13.45,20.34,Westpac,3.93,14.87,Aintree Food & Wi...,13.67,13.67,Rockbank Railway ...,9.64,55.86,weekly,490.0,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
Alfredton,https://www.domai...,59 Gillies Street...,-37.5612612,143.8173502,4,1,House,1,$390 p.w.,3350,390,Parkville College...,20.91,20.91,Creswick Hospital...,40.58,40.58,Ballarat Doctors,13.39,20.16,ANZ,11.68,11.85,Pizza Capers,2.92,17.06,Wendouree Railway...,13.11,18.47,weekly,390.0,2024,2,131.79681357598346,5.608938197685245,30.440857331100567,24.57777148129716,110.86720663375505,34.51507964534469,142.48033254591277,164.95636344350157,653.8970587950428,45.08382853852496,3.606027426382319,2.1996059669299344,36.85735461993613,46.78803544900097,15.80034954937439,9.1115689944728,61.91048606271024,233.3875951471112,0.6740030809956173,2.364753800736502,0.0,1.947809142106227,0.0,0.2403107786815788,0.0,1759.1121502065862,55.0,100.0,32.0,9.4,81.0,ballarat grammar,40.5,98.0,29.5,5.4,71.0,39.48424778470289,95.55052507384325,27.898949852313525,3.1529522120113755,57.40420059074589,109.3735516113442,124979.3,5698.7,"[[143.779068, -37...",120257500.0,15698750.0,0.010150094588695,0.1146602083030164,0.0057324906970459,0.0005301124669979,0.1305427935887574,0.8689270939442446
Altona Meadows,https://www.domai...,"6 Davis Court, Al...",-37.8780255,144.7792992,4,2,House,2,$650 per week,3028,650,Ozford College,9.3,9.3,Boardwalk Health ...,5.99,10.23,Unknown,8.05,8.05,Bendigo Bank,14.04,14.04,La Porchetta,19.32,29.96,Tarneit Railway S...,2.14,28.73,weekly,650.0,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,

In [36]:
partial_joined.describe()

summary,suburb,url,name,latitude,longitude,bedrooms,bathrooms,type,car_parks,cost_text,postcode,rent_pw,school_name,school_walking,school_driving,hospital_name,hospital_walking,hospital_driving,doctors_name,doctors_walking,doctors_driving,bank_name,bank_walking,bank_driving,restaurant_name,restaurant_walking,restaurant_driving,PTV_name,PTV_walking,PTV_driving,rate_type,cost_per_week,year,quarter,a20_assault_and_related_offences,a50_robbery,"a70_stalking,_harassment_and_threatening_behaviour",a80_dangerous_and_negligent_acts_endangering_people,other_crimes_against_the_person,b10_arson,b20_property_damage,b30_burglary/break_and_enter,b40_theft,b50_deception,c10_drug_dealing_and_trafficking,c20_cultivate_or_manufacture_drugs,c30_drug_use_and_possession,d10_weapons_and_explosives_offences,d20_disorderly_and_offensive_conduct,d30_public_nuisance_offences,e10_justice_procedures,e20_breaches_of_orders,f20_transport_regulation_offences,f90_miscellaneous_offences,c90_other_drug_offences,f30_other_government_regulatory_offences,d40_public_security_offences,f10_regulatory_driving_offences,b60_bribery,total_crimes,best_school_number_vce_subjects,best_school_satisfactory_complete_vce_percent,best_school_median_study_score,best_school_study_score_over_40_percent,best_school_percentage_applying_to_victorian_uni,best_school_school_name,avg_school_number_vce_subjects,avg_school_satisfactory_complete_vce_percent,avg_school_median_study_score,avg_school_study_score_over_40_percent,avg_school_percentage_applying_to_victorian_uni,zoned_school_number_vce_subjects,zoned_school_satisfactory_complete_vce_percent,zoned_school_median_study_score,zoned_school_study_score_over_40_percent,zoned_school_percentage_applying_to_victorian_uni,crow_distance_to_cbd,distance_to_cbd,duration_to_cbd,route_to_cbd,total_area,developed_area,built_percentage,urban_percentage,disturbed_percentage,water_percentage,developed_percentage,nature_percentage
count,9039,9039,9039,9039,9039,9039,9039,9039,9039,9039,9039,9039,9039,9039,9039,9039,9039,9039,9039,9039,9039,9039,9039,9039,9039,9039,9039,9039,9039,9039,9039,9039,3936,3936,3936,3936,3936,3936,3936,3936,3936,3936,3936,3936,3936,3936,3936,3936,3936,3936,3936,3936,3936,3936,3936,3936,3936,3936,3936,3936,3936,3936,3936,3936,3936,3936,3936,3936,3936,3936,3936,3936,3936,3936,3936,3936,3936,3936,3936,3936,3936,3936,3936,3936,3936,3936,3936,3936
mean,NULL,NULL,NULL,-37.779834033774264,144.89765160819172,2.684035844673083,1.573625401039938,NULL,1.376479699081757,560.6666666666666,3219.8607146808276,632.7031751299922,NULL,23.9919061843126,40.98838035180939,NULL,19.767883615443836,22.86282332116413,NULL,11.553330014382073,18.51511782276826,NULL,9.865818121473632,18.67415975218511,NULL,10.717206549397046,17.214622192720395,NULL,8.789481137293865,38.325453036840855,NULL,649.4961418084306,2024.0,2.0,228.83644035290072,18.658377517447466,59.97370405819383,26.20978100115208,74.98243228217522,10.904157800684274,212.65692460263412,212.63540613814192,1014.9183079718363,155.71953477678147,22.694701656840163,10.496794240769574,122.75353380688571,72.56371206318842,34.9330218660128,21.49216755625168,56.68126623194931,337.0653620005688,4.500243925880978,1.9918517729065597,1.600314256715109,2.135936710989042,0.6957872262296906,0.11883032768188541,3.668862510798839...,2705.21859014486,43.011686991869915,98.05160839990359,31.265861860265176,12.04278270797489,80.57457361258385,NULL,43.573518424420364,96.94517847405208,29.756737421502322,8.677057900473958,76.13232862576604,34.87190987724797,75.90959943415457,22.849718742176364,4.947739013625021,55.431492848879245,37.38687178992505,45050.18064024369,2606.7237296747894,NULL,4.9921913268546745E7,1.275705792682927E7,0.045730483504800114,0.6153689302693893,0.01853279876906894,0.007133368561450654,0.6796322125432578,0.3132344188952937
stddev,NULL,NULL,NULL,0.43405787696681347,0.551669308651965,1.0840740107410298,0.6308398157683571,NULL,1.0097905097827737,96.90675693794468,262.1360190869278,742.7530389405

In [37]:
partial_joined = partial_joined.drop("year", "quarter", "best_school_school_name",
                                     "route_to_cbd")

In [38]:
partial_joined_full = partial_joined.dropna()

In [39]:
partial_joined_full.limit(3)

suburb,url,name,latitude,longitude,bedrooms,bathrooms,type,car_parks,cost_text,postcode,rent_pw,school_name,school_walking,school_driving,hospital_name,hospital_walking,hospital_driving,doctors_name,doctors_walking,doctors_driving,bank_name,bank_walking,bank_driving,restaurant_name,restaurant_walking,restaurant_driving,PTV_name,PTV_walking,PTV_driving,rate_type,cost_per_week,a20_assault_and_related_offences,a50_robbery,"a70_stalking,_harassment_and_threatening_behaviour",a80_dangerous_and_negligent_acts_endangering_people,other_crimes_against_the_person,b10_arson,b20_property_damage,b30_burglary/break_and_enter,b40_theft,b50_deception,c10_drug_dealing_and_trafficking,c20_cultivate_or_manufacture_drugs,c30_drug_use_and_possession,d10_weapons_and_explosives_offences,d20_disorderly_and_offensive_conduct,d30_public_nuisance_offences,e10_justice_procedures,e20_breaches_of_orders,f20_transport_regulation_offences,f90_miscellaneous_offences,c90_other_drug_offences,f30_other_government_regulatory_offences,d40_public_security_offences,f10_regulatory_driving_offences,b60_bribery,total_crimes,best_school_number_vce_subjects,best_school_satisfactory_complete_vce_percent,best_school_median_study_score,best_school_study_score_over_40_percent,best_school_percentage_applying_to_victorian_uni,avg_school_number_vce_subjects,avg_school_satisfactory_complete_vce_percent,avg_school_median_study_score,avg_school_study_score_over_40_percent,avg_school_percentage_applying_to_victorian_uni,zoned_school_number_vce_subjects,zoned_school_satisfactory_complete_vce_percent,zoned_school_median_study_score,zoned_school_study_score_over_40_percent,zoned_school_percentage_applying_to_victorian_uni,crow_distance_to_cbd,distance_to_cbd,duration_to_cbd,total_area,developed_area,built_percentage,urban_percentage,disturbed_percentage,water_percentage,developed_percentage,nature_percentage
Albert Park,https://www.domai...,"8 Graham Street, ...",-37.8435292,144.9459585,4,3,Townhouse,2,$1600 per week,3206,1600,Ozford College,9.3,9.3,Peter MacCallum C...,28.42,34.4,nuGEN Medical & A...,15.1,14.95,ANZ,13.3,13.3,Hanoi Mee,14.42,14.42,Southern Cross Ra...,5.02,99.61,weekly,1600.0,193.72356466317277,17.57871728400593,56.50364392434726,22.259133314148805,98.11578342608074,3.9880208789769953,179.4308215269279,154.5100714296851,918.7136511150776,251.20118021952703,58.174847714297336,7.836771668748077,112.15705600660787,60.965884781128445,50.584491765696015,25.559150283130748,60.50053691440294,288.7700388420077,1.282853894517082,1.9363668638025688,0.1616815356166832,3.500012096453719,18.916739668308946,0.0083470330350795,0.0,2586.379366849703,57.0,99.0,31.0,6.3,83.0,57.0,99.0,31.0,6.3,83.0,50.954000614146445,98.48018046863834,30.9377258624176,6.76454713423704,78.99358325602229,4.53585711395057,6702.5,811.4,7043125.0,4928125.0,0.0247581861744609,0.6343952435886059,0.0405537314757298,0.0468542017925281,0.6997071612387967,0.2534386369686751
Albert Park,https://www.domai...,3/33 Bevan Street...,-37.8399588,144.9563729,1,1,Apartment / Unit ...,0,$575.00,3206,575,Ozford College,9.3,9.3,Epworth Richmond ...,10.78,10.78,Dr John Vissaritis,11.32,15.77,Bendigo Bank,18.85,18.85,Taco Bill,11.46,11.46,Southern Cross Ra...,5.02,99.61,weekly,575.0,193.72356466317277,17.57871728400593,56.50364392434726,22.259133314148805,98.11578342608074,3.9880208789769953,179.4308215269279,154.5100714296851,918.7136511150776,251.20118021952703,58.174847714297336,7.836771668748077,112.15705600660787,60.965884781128445,50.584491765696015,25.559150283130748,60.50053691440294,288.7700388420077,1.282853894517082,1.9363668638025688,0.1616815356166832,3.500012096453719,18.916739668308946,0.0083470330350795,0.0,2586.379366849703,57.0,99.0,31.0,6.3,83.0,57.0,99.0,31.0,6.3,83.0,50.954000614146445,98.48018046863834,30.9377258624176,6.76454713423704,78.99358325602229,4.53585711395057,6702.5,811.4,7043125.0,4928125.0,0.0247581861744609,0.6343952435886059,0.0405537314757298,0.0468542017925281,0.6997071612387967,0.2534386

In [40]:
partial_joined_full.count()

3936

In [41]:
partial_joined.write.csv("../../data/curated/partial_joined.csv", header=True, mode="overwrite")
partial_joined_full.write.csv("../../data/curated/partial_joined_full.csv", header=True, mode="overwrite")


**Full Economics Data By Malachy**

In [42]:
final_economics.limit(1)

suburbs,year,quarter,economic: median age of earners,economic: median income,economic: top 5% suburb,economic: top 1% suburb,economic: gini coefficient,economic: trimmed mean quarterly,economic: variable interest rate,economic: gdp quarterly,economic: gdp per capita quarterly,population: 0-4,population: 5-9,population: 10-14,population: 15-19,population: 20-24,population: 25-29,population: 30-34,population: 35-39,population: 40-44,population: 45-49,population: 50-54,population: 55-59,population: 60-64,population: 65-69,population: 70-74,population: 75-79,population: 80-84,population: 85 and over,population: total persons,population: growth,birth: brazil,birth: wales,birth: papua new guinea,birth: australia,birth: malaysia,birth: country of birth not stated,birth: indonesia,birth: egypt,birth: afghanistan age,birth: thailand,birth: ireland,birth: hong kong sar of china,birth: nepal,birth: north macedonia,birth: england,birth: chile,birth: fiji,birth: taiwan,birth: japan,birth: malta,birth: scotland,birth: china excludes sars and taiwan,birth: poland,birth: singapore,birth: south africa,birth: greece,birth: pakistan,birth: iraq,birth: mauritius,birth: samoa,birth: sri lanka,birth: india,birth: philippines,birth: vietnam,birth: new zealand,birth: iran,birth: lebanon,birth: france,birth: total,birth: netherlands,birth: myanmar,birth: united states of america,birth: bosnia and herzegovina,birth: zimbabwe,birth: bangladesh,birth: turkey,birth: canada,birth: korea republic of south,birth: cambodia,birth: born elsewhere,birth: italy,birth: croatia,birth: germany,studying: primary other,studying: tertiary FT 14-25,studying: tertiary FT 25+,studying: primary total,studying: tertiary PT 14-25,studying: tertiary PT 25+,studying: tafe total,studying: primary government,studying: secondary catholic,studying: preschool,studying: primary catholic,studying: secondary government,studying: secondary total,studying: tertiary total,overseas: 5 years,relationships: child under 15,relationships: defacto,relationships: other related individual,relationships: non dependent child,relationships: lone persons,relationships: group household,relationships: married,relationships: dependent student,relationships: lone parents,economic: number of earners,housing: type,housing: beds,housing: count,housing: median,housing: previous 2 years,housing: avg 2 years
Albert Park-Middl...,2002.0,1.0,NULL,NULL,30.75,14.55,0.5565,0.7,6.07,0.8,0.5,940.0,683.0,645.0,849.0,2020.0,3318.0,3379.0,2365.0,1780.0,1497.0,1479.0,1312.0,969.0,686.0,658.0,566.0,434.0,543.0,24123.0,1.0766231852810115,230.0,40.0,25.0,17208.0,330.0,1938.0,145.0,59.0,10.0,92.0,463.0,129.0,55.0,40.0,1467.0,71.0,28.0,63.0,113.0,30.0,205.0,711.0,128.0,109.0,225.0,389.0,37.0,4.0,47.0,4.0,93.0,819.0,197.0,126.0,769.0,145.0,17.0,208.0,29683.0,103.0,17.0,337.0,25.0,32.0,5.0,48.0,123.0,108.0,25.0,1611.0,241.0,35.0,217.0,221.0,742.0,506.0,1329.0,111.0,581.0,720.0,1023.0,97.0,424.0,83.0,548.0,1178.0,1939.0,3646.0,2914.0,4336.0,426.0,654.0,6156.0,1798.0,7732.0,994.0,761.0,0.0,all,all,1332.0,300.0,"[260.0, 270.0, 27...",275.7142857142857


In [43]:
final_economics.dtypes

[('suburbs', 'string'),
 ('year', 'string'),
 ('quarter', 'string'),
 ('economic: median age of earners', 'string'),
 ('economic: median income', 'string'),
 ('economic: top 5% suburb', 'string'),
 ('economic: top 1% suburb', 'string'),
 ('economic: gini coefficient', 'string'),
 ('economic: trimmed mean quarterly', 'string'),
 ('economic: variable interest rate', 'string'),
 ('economic: gdp quarterly', 'string'),
 ('economic: gdp per capita quarterly', 'string'),
 ('population: 0-4', 'string'),
 ('population: 5-9', 'string'),
 ('population: 10-14', 'string'),
 ('population: 15-19', 'string'),
 ('population: 20-24', 'string'),
 ('population: 25-29', 'string'),
 ('population: 30-34', 'string'),
 ('population: 35-39', 'string'),
 ('population: 40-44', 'string'),
 ('population: 45-49', 'string'),
 ('population: 50-54', 'string'),
 ('population: 55-59', 'string'),
 ('population: 60-64', 'string'),
 ('population: 65-69', 'string'),
 ('population: 70-74', 'string'),
 ('population: 75-79', 's

In [44]:
# Count unique items in the 'letter' column
unique_count = final_economics.select('suburbs').distinct().count()
print(f"Number of unique items in 'suburbs' column: {unique_count}")

Number of unique items in 'suburbs' column: 144


In [45]:
# Filtering by year
final_economics = final_economics.filter((F.col('year') == 2023) & (F.col("housing: type") == "all"))
final_economics = final_economics.drop('economic: median age of earners',
                                       'economic: median income',
                                       'housing: type',
                                       'housing: beds',
                                       'year',
                                       'quarter',
                                       'housing: previous 2 years')

# Show the filtered DataFrame
final_economics.limit(3)


suburbs,economic: top 5% suburb,economic: top 1% suburb,economic: gini coefficient,economic: trimmed mean quarterly,economic: variable interest rate,economic: gdp quarterly,economic: gdp per capita quarterly,population: 0-4,population: 5-9,population: 10-14,population: 15-19,population: 20-24,population: 25-29,population: 30-34,population: 35-39,population: 40-44,population: 45-49,population: 50-54,population: 55-59,population: 60-64,population: 65-69,population: 70-74,population: 75-79,population: 80-84,population: 85 and over,population: total persons,population: growth,birth: brazil,birth: wales,birth: papua new guinea,birth: australia,birth: malaysia,birth: country of birth not stated,birth: indonesia,birth: egypt,birth: afghanistan age,birth: thailand,birth: ireland,birth: hong kong sar of china,birth: nepal,birth: north macedonia,birth: england,birth: chile,birth: fiji,birth: taiwan,birth: japan,birth: malta,birth: scotland,birth: china excludes sars and taiwan,birth: poland,birth: singapore,birth: south africa,birth: greece,birth: pakistan,birth: iraq,birth: mauritius,birth: samoa,birth: sri lanka,birth: india,birth: philippines,birth: vietnam,birth: new zealand,birth: iran,birth: lebanon,birth: france,birth: total,birth: netherlands,birth: myanmar,birth: united states of america,birth: bosnia and herzegovina,birth: zimbabwe,birth: bangladesh,birth: turkey,birth: canada,birth: korea republic of south,birth: cambodia,birth: born elsewhere,birth: italy,birth: croatia,birth: germany,studying: primary other,studying: tertiary FT 14-25,studying: tertiary FT 25+,studying: primary total,studying: tertiary PT 14-25,studying: tertiary PT 25+,studying: tafe total,studying: primary government,studying: secondary catholic,studying: preschool,studying: primary catholic,studying: secondary government,studying: secondary total,studying: tertiary total,overseas: 5 years,relationships: child under 15,relationships: defacto,relationships: other related individual,relationships: non dependent child,relationships: lone persons,relationships: group household,relationships: married,relationships: dependent student,relationships: lone parents,economic: number of earners,housing: count,housing: median,housing: avg 2 years
Albert Park-Middl...,30.75,14.55,0.5565,1.2,7.77,0.5,-0.1,1060.0,982.0,1100.0,1192.0,2138.0,3800.0,3710.0,2928.0,2312.0,2038.0,2226.0,2011.0,1833.0,1433.0,1231.0,1124.0,645.0,578.0,32341.0,3.1541015462808746,230.0,40.0,25.0,17208.0,330.0,1938.0,145.0,59.0,10.0,92.0,463.0,129.0,55.0,40.0,1467.0,71.0,28.0,63.0,113.0,30.0,205.0,711.0,128.0,109.0,225.0,389.0,37.0,4.0,47.0,4.0,93.0,819.0,197.0,126.0,769.0,145.0,17.0,208.0,29683.0,103.0,17.0,337.0,25.0,32.0,5.0,48.0,123.0,108.0,25.0,1611.0,241.0,35.0,217.0,221.0,742.0,506.0,1329.0,111.0,581.0,720.0,1023.0,97.0,424.0,83.0,548.0,1178.0,1939.0,3646.0,2914.0,4336.0,426.0,654.0,6156.0,1798.0,7732.0,994.0,761.0,0.0,795.0,545.0,506.1428571428572
Albert Park-Middl...,30.75,14.55,0.5565,0.9,8.27,0.5,-0.1,1060.0,982.0,1100.0,1192.0,2138.0,3800.0,3710.0,2928.0,2312.0,2038.0,2226.0,2011.0,1833.0,1433.0,1231.0,1124.0,645.0,578.0,32341.0,3.1541015462808746,230.0,40.0,25.0,17208.0,330.0,1938.0,145.0,59.0,10.0,92.0,463.0,129.0,55.0,40.0,1467.0,71.0,28.0,63.0,113.0,30.0,205.0,711.0,128.0,109.0,225.0,389.0,37.0,4.0,47.0,4.0,93.0,819.0,197.0,126.0,769.0,145.0,17.0,208.0,29683.0,103.0,17.0,337.0,25.0,32.0,5.0,48.0,123.0,108.0,25.0,1611.0,241.0,35.0,217.0,221.0,742.0,506.0,1329.0,111.0,581.0,720.0,1023.0,97.0,424.0,83.0,548.0,1178.0,1939.0,3646.0,2914.0,4336.0,426.0,654.0,6156.0,1798.0,7732.0,994.0,761.0,0.0,739.0,550.0,512.5714285714286
Albert Park-Middl...,30.75,14.55,0.5565,1.2,8.52,0.3,-0.4,1060.0,982.0,1100.0,1192.0,2138.0,3800.0,3710.0,2928.0,2312.0,2038.0,2226.0,2011.0,1833.0,1433.0,1231.0,1124.0,645.0,578.0,32341.0,3.1541015462808746,230.0,40.0,25.0,17208.0,330.0,1938.0,145.0,59.0,10.0,92.0,463.0,129.0,55.0,40.0,1467.0,71.0,28.0,63.0,113.0,30.0,205.0,711.0,128.0,109.0,225.0,389.0,37.0,4.0,47.0,

In [46]:
# Identify string columns
string_columns = final_economics.drop("suburbs").columns

# Cast all string columns containing numerical values to float
for col in string_columns:
    final_economics = final_economics.withColumn(col, F.col(col).cast("float"))


In [47]:
# Identify numerical columns
numerical_columns = final_economics.drop('suburbs').columns

# Create aggregation expressions for mean
agg_exprs = [F.mean(col).alias(f"mean_{col}") for col in numerical_columns]

# Group by suburb and calculate the mean for all numerical columns
mean_df = final_economics.groupBy("suburbs").agg(*agg_exprs)

# Show the result
mean_df.limit(2)

suburbs,mean_economic: top 5% suburb,mean_economic: top 1% suburb,mean_economic: gini coefficient,mean_economic: trimmed mean quarterly,mean_economic: variable interest rate,mean_economic: gdp quarterly,mean_economic: gdp per capita quarterly,mean_population: 0-4,mean_population: 5-9,mean_population: 10-14,mean_population: 15-19,mean_population: 20-24,mean_population: 25-29,mean_population: 30-34,mean_population: 35-39,mean_population: 40-44,mean_population: 45-49,mean_population: 50-54,mean_population: 55-59,mean_population: 60-64,mean_population: 65-69,mean_population: 70-74,mean_population: 75-79,mean_population: 80-84,mean_population: 85 and over,mean_population: total persons,mean_population: growth,mean_birth: brazil,mean_birth: wales,mean_birth: papua new guinea,mean_birth: australia,mean_birth: malaysia,mean_birth: country of birth not stated,mean_birth: indonesia,mean_birth: egypt,mean_birth: afghanistan age,mean_birth: thailand,mean_birth: ireland,mean_birth: hong kong sar of china,mean_birth: nepal,mean_birth: north macedonia,mean_birth: england,mean_birth: chile,mean_birth: fiji,mean_birth: taiwan,mean_birth: japan,mean_birth: malta,mean_birth: scotland,mean_birth: china excludes sars and taiwan,mean_birth: poland,mean_birth: singapore,mean_birth: south africa,mean_birth: greece,mean_birth: pakistan,mean_birth: iraq,mean_birth: mauritius,mean_birth: samoa,mean_birth: sri lanka,mean_birth: india,mean_birth: philippines,mean_birth: vietnam,mean_birth: new zealand,mean_birth: iran,mean_birth: lebanon,mean_birth: france,mean_birth: total,mean_birth: netherlands,mean_birth: myanmar,mean_birth: united states of america,mean_birth: bosnia and herzegovina,mean_birth: zimbabwe,mean_birth: bangladesh,mean_birth: turkey,mean_birth: canada,mean_birth: korea republic of south,mean_birth: cambodia,mean_birth: born elsewhere,mean_birth: italy,mean_birth: croatia,mean_birth: germany,mean_studying: primary other,mean_studying: tertiary FT 14-25,mean_studying: tertiary FT 25+,mean_studying: primary total,mean_studying: tertiary PT 14-25,mean_studying: tertiary PT 25+,mean_studying: tafe total,mean_studying: primary government,mean_studying: secondary catholic,mean_studying: preschool,mean_studying: primary catholic,mean_studying: secondary government,mean_studying: secondary total,mean_studying: tertiary total,mean_overseas: 5 years,mean_relationships: child under 15,mean_relationships: defacto,mean_relationships: other related individual,mean_relationships: non dependent child,mean_relationships: lone persons,mean_relationships: group household,mean_relationships: married,mean_relationships: dependent student,mean_relationships: lone parents,mean_economic: number of earners,mean_housing: count,mean_housing: median,mean_housing: avg 2 years
Brighton,39.5,19.200000762939453,0.6480000019073486,1.0250000208616257,8.31166684627533,0.3750000037252903,-0.22500000521540642,871.0,1145.0,1499.0,1664.0,1381.0,864.0,961.0,1150.0,1337.0,1524.0,1802.0,1691.0,1709.0,1432.0,1404.0,1323.0,863.0,927.0,23547.0,0.6782987713813782,42.0,31.0,6.0,15725.0,126.0,791.0,46.0,65.0,9.0,44.0,142.0,93.0,12.0,21.0,1236.0,28.0,9.0,24.0,79.0,15.0,156.0,802.0,111.0,85.0,265.0,116.0,13.0,5.0,19.0,7.0,62.0,168.0,52.0,75.0,359.0,38.0,21.0,84.0,22758.0,73.0,9.0,241.0,10.0,16.0,7.0,31.0,78.0,36.0,12.0,905.0,181.0,34.0,155.0,606.0,926.0,154.0,1687.0,90.0,245.0,230.0,734.0,287.0,409.0,345.0,232.0,1784.0,1416.0,1513.0,3325.0,1610.0,271.0,1006.0,2455.0,331.0,9112.0,1804.0,760.0,0.0,794.5,761.25,681.2142791748047
Bendigo,22.100000381469727,9.399999618530273,0.4740000069141388,1.0250000208616257,8.31166684627533,0.3750000037252903,-0.22500000521540642,813.0,702.0,724.0,771.0,1003.0,1263.0,1188.0,889.0,786.0,799.0,947.0,933.0,907.0,828.0,751.0,570.0,347.0,404.0,14625.0,-0.2085028439760208,12.0,14.0,16.0,12151.0,39.0,990.0,7.0,8.0,11.0,161.0,30.0,16.0,34.0,0.0,249.0,4.0,13.0,10.0,5.0,6.0,40.0,62.0,7.0,9.0,25.0,11.0,20.0,0.0,0.0,0.0,35.0,266.0,111.0,20.0,106.0,12.0,4.0,6.0,14929.

In [48]:
final_economics = splitting_suburb(mean_df)
final_economics = final_economics.dropna()
partial_joined_part_2 = partial_joined.join(final_economics, on="suburb", how="left")

partial_joined_part_2_full = partial_joined_part_2.dropna()


In [49]:
partial_joined_part_2.count()

9039

In [50]:
partial_joined_part_2.limit(1)

suburb,url,name,latitude,longitude,bedrooms,bathrooms,type,car_parks,cost_text,postcode,rent_pw,school_name,school_walking,school_driving,hospital_name,hospital_walking,hospital_driving,doctors_name,doctors_walking,doctors_driving,bank_name,bank_walking,bank_driving,restaurant_name,restaurant_walking,restaurant_driving,PTV_name,PTV_walking,PTV_driving,rate_type,cost_per_week,a20_assault_and_related_offences,a50_robbery,"a70_stalking,_harassment_and_threatening_behaviour",a80_dangerous_and_negligent_acts_endangering_people,other_crimes_against_the_person,b10_arson,b20_property_damage,b30_burglary/break_and_enter,b40_theft,b50_deception,c10_drug_dealing_and_trafficking,c20_cultivate_or_manufacture_drugs,c30_drug_use_and_possession,d10_weapons_and_explosives_offences,d20_disorderly_and_offensive_conduct,d30_public_nuisance_offences,e10_justice_procedures,e20_breaches_of_orders,f20_transport_regulation_offences,f90_miscellaneous_offences,c90_other_drug_offences,f30_other_government_regulatory_offences,d40_public_security_offences,f10_regulatory_driving_offences,b60_bribery,total_crimes,best_school_number_vce_subjects,best_school_satisfactory_complete_vce_percent,best_school_median_study_score,best_school_study_score_over_40_percent,best_school_percentage_applying_to_victorian_uni,avg_school_number_vce_subjects,avg_school_satisfactory_complete_vce_percent,avg_school_median_study_score,avg_school_study_score_over_40_percent,avg_school_percentage_applying_to_victorian_uni,zoned_school_number_vce_subjects,zoned_school_satisfactory_complete_vce_percent,zoned_school_median_study_score,zoned_school_study_score_over_40_percent,zoned_school_percentage_applying_to_victorian_uni,crow_distance_to_cbd,distance_to_cbd,duration_to_cbd,total_area,developed_area,built_percentage,urban_percentage,disturbed_percentage,water_percentage,developed_percentage,nature_percentage,mean_economic: top 5% suburb,mean_economic: top 1% suburb,mean_economic: gini coefficient,mean_economic: trimmed mean quarterly,mean_economic: variable interest rate,mean_economic: gdp quarterly,mean_economic: gdp per capita quarterly,mean_population: 0-4,mean_population: 5-9,mean_population: 10-14,mean_population: 15-19,mean_population: 20-24,mean_population: 25-29,mean_population: 30-34,mean_population: 35-39,mean_population: 40-44,mean_population: 45-49,mean_population: 50-54,mean_population: 55-59,mean_population: 60-64,mean_population: 65-69,mean_population: 70-74,mean_population: 75-79,mean_population: 80-84,mean_population: 85 and over,mean_population: total persons,mean_population: growth,mean_birth: brazil,mean_birth: wales,mean_birth: papua new guinea,mean_birth: australia,mean_birth: malaysia,mean_birth: country of birth not stated,mean_birth: indonesia,mean_birth: egypt,mean_birth: afghanistan age,mean_birth: thailand,mean_birth: ireland,mean_birth: hong kong sar of china,mean_birth: nepal,mean_birth: north macedonia,mean_birth: england,mean_birth: chile,mean_birth: fiji,mean_birth: taiwan,mean_birth: japan,mean_birth: malta,mean_birth: scotland,mean_birth: china excludes sars and taiwan,mean_birth: poland,mean_birth: singapore,mean_birth: south africa,mean_birth: greece,mean_birth: pakistan,mean_birth: iraq,mean_birth: mauritius,mean_birth: samoa,mean_birth: sri lanka,mean_birth: india,mean_birth: philippines,mean_birth: vietnam,mean_birth: new zealand,mean_birth: iran,mean_birth: lebanon,mean_birth: france,mean_birth: total,mean_birth: netherlands,mean_birth: myanmar,mean_birth: united states of america,mean_birth: bosnia and herzegovina,mean_birth: zimbabwe,mean_birth: bangladesh,mean_birth: turkey,mean_birth: canada,mean_birth: korea republic of south,mean_birth: cambodia,mean_birth: born elsewhere,mean_birth: italy,mean_birth: croatia,mean_birth: germany,mean_studying: primary other,mean_studying: tertiary FT 14-25,mean_studying: tertiary FT 25+,mean_studying: primary total,mean_studying: tertiary PT 14-25,mean_studying: tertiary PT 25+,mean_studying: tafe total,m

In [51]:
partial_joined_part_2_full.count()

3936

In [52]:
partial_joined_part_2.write.csv("../../data/curated/partial_joined_part_2.csv", header=True, mode="overwrite")
partial_joined_part_2_full.write.csv("../../data/curated/partial_joined_part_2_full.csv", header=True, mode="overwrite")

**Final Data by Jun Merged by suburbs**

In [53]:
merged_suburb = merged_suburb.drop("_c0")

In [54]:
merged_suburb.limit(3)

suburbs,year,economic: median income,economic: median age of earners,economic: gini coefficient,economic: top 1% suburb,economic: top 5% suburb,economic: trimmed mean quarterly,economic: variable interest rate,economic: gdp quarterly,economic: gdp per capita quarterly,economic: number of earners,population: total,birth: total,birth: taiwan,birth: country of birth not stated,birth: croatia,studying: primary other,birth: poland,studying: tertiary FT 14-25,birth: egypt,birth: singapore,birth: germany,relationships: dependent student,studying: secondary government,birth: nepal,studying: tafe total,birth: philippines,birth: greece,relationships: non dependent child,birth: pakistan,birth: iraq,birth: mauritius,ages: 55-64,birth: netherlands,birth: north macedonia,birth: thailand,birth: afghanistan age,birth: china excludes sars and taiwan,overseas: 5 years,studying: preschool,birth: samoa,birth: malaysia,studying: primary government,ages: 35-44,birth: japan,birth: myanmar,studying: primary total,birth: scotland,ages: 25-34,birth: canada,ages: 45-54,birth: malta,birth: vietnam,relationships: lone persons,birth: sri lanka,birth: indonesia,birth: fiji,studying: tertiary PT 14-25,relationships: married,relationships: child under 15,studying: tertiary PT 25+,ages: 15-24,ages: 0-14,birth: england,birth: south africa,birth: chile,birth: italy,studying: secondary catholic,relationships: group household,birth: wales,birth: korea republic of south,ages: 65-74,birth: bangladesh,birth: iran,studying: tertiary total,birth: papua new guinea,studying: tertiary FT 25+,studying: primary catholic,studying: secondary total,birth: turkey,birth: zimbabwe,relationships: defacto,birth: india,birth: france,birth: new zealand,birth: born elsewhere,relationships: other related individual,birth: united states of america,ages: 75-84,birth: lebanon,birth: bosnia and herzegovina,birth: ireland,birth: cambodia,relationships: lone parents,birth: australia,birth: hong kong sar of china,population: growth,birth: brazil
Albert Park-Middl...,2016,58937.5,39.0,0.5565,14.55,30.75,0.35,5.281666666666666,0.6,0.14999999999999997,42752,63284.0,59366,126,3876,70,442,256,1484,118,218,434,1988,1096,110,1440,394,778,1308,74,8,94,6530,206,80,184,20,1422,7292,848,8,660,2046,9160,226,34,2658,410,11188,246,7596,60,252,12312,186,290,56,222,15464,5828,1162,4658,5900,2934,450,142,482,194,3596,80,216,4630,10,290,3878,50,1012,166,2356,96,64,8672,1638,416,1538,3222,852,674,2620,34,50,926,50,1522,34416,258,2.591843919235548,460
Albert Park-Middl...,2017,60361.75,39.25,0.5565,14.55,30.75,0.45,5.243333333333333,0.55,0.175,86486,127512.0,118732,252,7752,140,884,512,2968,236,436,868,3976,2192,220,2880,788,1556,2616,148,16,188,13060,412,160,368,40,2844,14584,1696,16,1320,4092,18320,452,68,5316,820,22376,492,15192,120,504,24624,372,580,112,444,30928,11656,2324,9316,11800,5868,900,284,964,388,7192,160,432,9260,20,580,7756,100,2024,332,4712,192,128,17344,3276,832,3076,6444,1704,1348,5240,68,100,1852,100,3044,68832,516,1.5521950841539178,920
Albert Park-Middl...,2018,62675.0,39.5,0.5565,14.55,30.75,0.42500000000000004,5.2575,0.575,0.25,88506,129512.0,118732,252,7752,140,884,512,2968,236,436,868,3976,2192,220,2880,788,1556,2616,148,16,188,13060,412,160,368,40,2844,14584,1696,16,1320,4092,18320,452,68,5316,820,22376,492,15192,120,504,24624,372,580,112,444,30928,11656,2324,9316,11800,5868,900,284,964,388,7192,160,432,9260,20,580,7756,100,2024,332,4712,192,128,17344,3276,832,3076,6444,1704,1348,5240,68,100,1852,100,3044,68832,516,3.1505787488619985,920


In [55]:
merged_suburb.describe()

summary,suburbs,year,economic: median income,economic: median age of earners,economic: gini coefficient,economic: top 1% suburb,economic: top 5% suburb,economic: trimmed mean quarterly,economic: variable interest rate,economic: gdp quarterly,economic: gdp per capita quarterly,economic: number of earners,population: total,birth: total,birth: taiwan,birth: country of birth not stated,birth: croatia,studying: primary other,birth: poland,studying: tertiary FT 14-25,birth: egypt,birth: singapore,birth: germany,relationships: dependent student,studying: secondary government,birth: nepal,studying: tafe total,birth: philippines,birth: greece,relationships: non dependent child,birth: pakistan,birth: iraq,birth: mauritius,ages: 55-64,birth: netherlands,birth: north macedonia,birth: thailand,birth: afghanistan age,birth: china excludes sars and taiwan,overseas: 5 years,studying: preschool,birth: samoa,birth: malaysia,studying: primary government,ages: 35-44,birth: japan,birth: myanmar,studying: primary total,birth: scotland,ages: 25-34,birth: canada,ages: 45-54,birth: malta,birth: vietnam,relationships: lone persons,birth: sri lanka,birth: indonesia,birth: fiji,studying: tertiary PT 14-25,relationships: married,relationships: child under 15,studying: tertiary PT 25+,ages: 15-24,ages: 0-14,birth: england,birth: south africa,birth: chile,birth: italy,studying: secondary catholic,relationships: group household,birth: wales,birth: korea republic of south,ages: 65-74,birth: bangladesh,birth: iran,studying: tertiary total,birth: papua new guinea,studying: tertiary FT 25+,studying: primary catholic,studying: secondary total,birth: turkey,birth: zimbabwe,relationships: defacto,birth: india,birth: france,birth: new zealand,birth: born elsewhere,relationships: other related individual,birth: united states of america,ages: 75-84,birth: lebanon,birth: bosnia and herzegovina,birth: ireland,birth: cambodia,relationships: lone parents,birth: australia,birth: hong kong sar of china,population: growth,birth: brazil
count,864,864,864,864,864,864,864,864,864,864,864,864,864,864,864,864,864,864,864,864,864,864,864,864,864,864,864,864,864,864,864,864,864,864,864,864,864,864,864,864,864,864,864,864,864,864,864,864,864,864,864,864,864,864,864,864,864,864,864,864,864,864,864,864,864,864,864,864,864,864,864,864,864,864,864,864,864,864,864,864,864,864,864,864,864,864,864,864,864,864,864,864,864,864,864,864,864,863,864
mean,NULL,2018.5,53563.20515459655,41.172625936948855,0.4723998346560845,8.319201388888883,21.279216269841257,0.38333333333333436,4.995138888888882,0.6333333333333341,0.34166666666666756,57636.33101851852,100485.32175925926,99110.74074074074,203.84259259259258,4636.643518518518,239.93055555555554,868.1712962962963,244.35185185185185,2969.421296296296,219.65277777777777,323.05555555555554,398.8425925925926,5152.615740740741,3357.314814814815,366.89814814814815,2444.953703703704,978.6111111111111,896.3888888888889,5898.449074074074,464.35185185185185,413.14814814814815,200.8101851851852,10418.587962962964,257.22222222222223,287.1296296296296,358.19444444444446,455.2083333333333,3386.7361111111113,6652.407407407408,2040.0,108.26388888888889,1192.4074074074074,5113.981481481482,13315.717592592593,178.10185185185185,229.8148148148148,7498.888888888889,404.212962962963,14924.837962962964,181.22685185185185,11930.50925925926,230.94907407407408,1575.601851851852,10371.736111111111,1145.9490740740741,371.89814814814815,182.0601851851852,356.36574074074076,33615.94907407407,15982.291666666666,1205.3009259259259,11332.61574074074,16147.037037037036,2680.4861111111113,515.0462962962963,134.39814814814815,1139.4675925925926,1397.9166666666667,4277.731481481482,74.76851851851852,300.18518518518516,8150.925925925926,146.2962962962963,394.81481481481484,5741.597222222223,42.5462962962963,1200.625,1506.4351851851852,6121.342592592592,273.10185185185185,87.52314814814815,8583.958333333334,3574.0046296296296,151.66666666666666,1535.601851851852,3459.1203703703704,2291

In [56]:
latest_year_df = merged_suburb.groupBy("suburbs").agg(
    F.max("year").alias("latest_year")
)

# Step 2: Join back with the original DataFrame to get rows for the latest year
latest_suburbs_df = merged_suburb.join(
    latest_year_df,
    (merged_suburb.suburbs == latest_year_df.suburbs) & (merged_suburb.year == latest_year_df.latest_year),
    "inner"
).drop(latest_year_df.latest_year, latest_year_df.suburbs)

latest_suburbs_df.limit(3)

year,economic: median income,economic: median age of earners,economic: gini coefficient,economic: top 1% suburb,economic: top 5% suburb,economic: trimmed mean quarterly,economic: variable interest rate,economic: gdp quarterly,economic: gdp per capita quarterly,economic: number of earners,population: total,birth: total,birth: taiwan,birth: country of birth not stated,birth: croatia,studying: primary other,birth: poland,studying: tertiary FT 14-25,birth: egypt,birth: singapore,birth: germany,relationships: dependent student,studying: secondary government,birth: nepal,studying: tafe total,birth: philippines,birth: greece,relationships: non dependent child,birth: pakistan,birth: iraq,birth: mauritius,ages: 55-64,birth: netherlands,birth: north macedonia,birth: thailand,birth: afghanistan age,birth: china excludes sars and taiwan,overseas: 5 years,studying: preschool,birth: samoa,birth: malaysia,studying: primary government,ages: 35-44,birth: japan,birth: myanmar,studying: primary total,birth: scotland,ages: 25-34,birth: canada,ages: 45-54,birth: malta,birth: vietnam,relationships: lone persons,birth: sri lanka,birth: indonesia,birth: fiji,studying: tertiary PT 14-25,relationships: married,relationships: child under 15,studying: tertiary PT 25+,ages: 15-24,ages: 0-14,birth: england,birth: south africa,birth: chile,birth: italy,studying: secondary catholic,relationships: group household,birth: wales,birth: korea republic of south,ages: 65-74,birth: bangladesh,birth: iran,studying: tertiary total,birth: papua new guinea,studying: tertiary FT 25+,studying: primary catholic,studying: secondary total,birth: turkey,birth: zimbabwe,relationships: defacto,birth: india,birth: france,birth: new zealand,birth: born elsewhere,relationships: other related individual,birth: united states of america,ages: 75-84,birth: lebanon,birth: bosnia and herzegovina,birth: ireland,birth: cambodia,relationships: lone parents,birth: australia,birth: hong kong sar of china,population: growth,birth: brazil,suburbs
2021,68526.5,41.0,0.5565,14.55,30.75,0.4,4.52,1.55,1.4500000000000002,41460,60202.0,59366,126,3876,70,442,256,1484,118,218,434,1988,1096,110,1440,394,778,1308,74,8,94,6530,206,80,184,20,1422,7292,848,8,660,2046,9160,226,34,2658,410,11188,246,7596,60,252,12312,186,290,56,222,15464,5828,1162,4658,5900,2934,450,142,482,194,3596,80,216,4630,10,290,3878,50,1012,166,2356,96,64,8672,1638,416,1538,3222,852,674,2620,34,50,926,50,1522,34416,258,-7.762111558986584,460,Albert Park-Middl...
2021,63293.0,41.0,0.434,6.15,17.65,0.4,4.52,1.55,1.4500000000000002,31850,57478.0,56974,40,2948,416,402,200,992,76,112,380,2406,1360,54,1062,490,894,3428,118,42,32,5344,154,414,164,22,508,2396,1348,18,238,2950,8708,98,66,4512,364,7816,112,7140,924,1300,6462,116,282,54,154,19212,9862,636,4662,9980,1846,256,52,1326,1108,1394,58,72,4638,42,118,2222,14,442,1168,2900,56,30,5776,904,118,1200,2212,1262,204,3586,1050,76,356,32,2548,36000,116,-1.8166800838888109,192,Altona
2021,73321.0,41.0,0.569,13.9,31.6,0.4,4.52,1.55,1.4500000000000002,13308,18866.0,18672,34,734,28,414,106,714,28,74,114,788,100,32,312,82,196,550,24,10,26,1666,34,6,120,6,592,1440,336,0,174,376,2598,72,0,980,60,3678,78,2196,12,86,3434,96,72,16,102,5610,2196,352,1850,2250,660,188,32,108,164,1142,12,46,1648,12,52,1428,0,256,186,844,24,12,2370,424,62,328,692,298,202,1178,24,8,98,10,610,12560,134,-3.186104218362285,40,Armadale


In [57]:
merged_suburb = splitting_suburb(latest_suburbs_df)

In [58]:
merged_suburb = merged_suburb.drop('year')
merged_suburb.limit(3)

economic: median income,economic: median age of earners,economic: gini coefficient,economic: top 1% suburb,economic: top 5% suburb,economic: trimmed mean quarterly,economic: variable interest rate,economic: gdp quarterly,economic: gdp per capita quarterly,economic: number of earners,population: total,birth: total,birth: taiwan,birth: country of birth not stated,birth: croatia,studying: primary other,birth: poland,studying: tertiary FT 14-25,birth: egypt,birth: singapore,birth: germany,relationships: dependent student,studying: secondary government,birth: nepal,studying: tafe total,birth: philippines,birth: greece,relationships: non dependent child,birth: pakistan,birth: iraq,birth: mauritius,ages: 55-64,birth: netherlands,birth: north macedonia,birth: thailand,birth: afghanistan age,birth: china excludes sars and taiwan,overseas: 5 years,studying: preschool,birth: samoa,birth: malaysia,studying: primary government,ages: 35-44,birth: japan,birth: myanmar,studying: primary total,birth: scotland,ages: 25-34,birth: canada,ages: 45-54,birth: malta,birth: vietnam,relationships: lone persons,birth: sri lanka,birth: indonesia,birth: fiji,studying: tertiary PT 14-25,relationships: married,relationships: child under 15,studying: tertiary PT 25+,ages: 15-24,ages: 0-14,birth: england,birth: south africa,birth: chile,birth: italy,studying: secondary catholic,relationships: group household,birth: wales,birth: korea republic of south,ages: 65-74,birth: bangladesh,birth: iran,studying: tertiary total,birth: papua new guinea,studying: tertiary FT 25+,studying: primary catholic,studying: secondary total,birth: turkey,birth: zimbabwe,relationships: defacto,birth: india,birth: france,birth: new zealand,birth: born elsewhere,relationships: other related individual,birth: united states of america,ages: 75-84,birth: lebanon,birth: bosnia and herzegovina,birth: ireland,birth: cambodia,relationships: lone parents,birth: australia,birth: hong kong sar of china,population: growth,birth: brazil,suburb
68526.5,41.0,0.5565,14.55,30.75,0.4,4.52,1.55,1.4500000000000002,41460,60202.0,59366,126,3876,70,442,256,1484,118,218,434,1988,1096,110,1440,394,778,1308,74,8,94,6530,206,80,184,20,1422,7292,848,8,660,2046,9160,226,34,2658,410,11188,246,7596,60,252,12312,186,290,56,222,15464,5828,1162,4658,5900,2934,450,142,482,194,3596,80,216,4630,10,290,3878,50,1012,166,2356,96,64,8672,1638,416,1538,3222,852,674,2620,34,50,926,50,1522,34416,258,-7.762111558986584,460,Albert Park
68526.5,41.0,0.5565,14.55,30.75,0.4,4.52,1.55,1.4500000000000002,41460,60202.0,59366,126,3876,70,442,256,1484,118,218,434,1988,1096,110,1440,394,778,1308,74,8,94,6530,206,80,184,20,1422,7292,848,8,660,2046,9160,226,34,2658,410,11188,246,7596,60,252,12312,186,290,56,222,15464,5828,1162,4658,5900,2934,450,142,482,194,3596,80,216,4630,10,290,3878,50,1012,166,2356,96,64,8672,1638,416,1538,3222,852,674,2620,34,50,926,50,1522,34416,258,-7.762111558986584,460,Middle Park
68526.5,41.0,0.5565,14.55,30.75,0.4,4.52,1.55,1.4500000000000002,41460,60202.0,59366,126,3876,70,442,256,1484,118,218,434,1988,1096,110,1440,394,778,1308,74,8,94,6530,206,80,184,20,1422,7292,848,8,660,2046,9160,226,34,2658,410,11188,246,7596,60,252,12312,186,290,56,222,15464,5828,1162,4658,5900,2934,450,142,482,194,3596,80,216,4630,10,290,3878,50,1012,166,2356,96,64,8672,1638,416,1538,3222,852,674,2620,34,50,926,50,1522,34416,258,-7.762111558986584,460,West St Kilda


In [59]:
finished_df = partial_joined_part_2.join(merged_suburb, on="suburb", how="left")
finished_df_full = finished_df.dropna()

In [60]:
finished_df.count()

9039

In [61]:
finished_df_full.count()

3936

In [62]:
finished_df_full.limit(3)

suburb,url,name,latitude,longitude,bedrooms,bathrooms,type,car_parks,cost_text,postcode,rent_pw,school_name,school_walking,school_driving,hospital_name,hospital_walking,hospital_driving,doctors_name,doctors_walking,doctors_driving,bank_name,bank_walking,bank_driving,restaurant_name,restaurant_walking,restaurant_driving,PTV_name,PTV_walking,PTV_driving,rate_type,cost_per_week,a20_assault_and_related_offences,a50_robbery,"a70_stalking,_harassment_and_threatening_behaviour",a80_dangerous_and_negligent_acts_endangering_people,other_crimes_against_the_person,b10_arson,b20_property_damage,b30_burglary/break_and_enter,b40_theft,b50_deception,c10_drug_dealing_and_trafficking,c20_cultivate_or_manufacture_drugs,c30_drug_use_and_possession,d10_weapons_and_explosives_offences,d20_disorderly_and_offensive_conduct,d30_public_nuisance_offences,e10_justice_procedures,e20_breaches_of_orders,f20_transport_regulation_offences,f90_miscellaneous_offences,c90_other_drug_offences,f30_other_government_regulatory_offences,d40_public_security_offences,f10_regulatory_driving_offences,b60_bribery,total_crimes,best_school_number_vce_subjects,best_school_satisfactory_complete_vce_percent,best_school_median_study_score,best_school_study_score_over_40_percent,best_school_percentage_applying_to_victorian_uni,avg_school_number_vce_subjects,avg_school_satisfactory_complete_vce_percent,avg_school_median_study_score,avg_school_study_score_over_40_percent,avg_school_percentage_applying_to_victorian_uni,zoned_school_number_vce_subjects,zoned_school_satisfactory_complete_vce_percent,zoned_school_median_study_score,zoned_school_study_score_over_40_percent,zoned_school_percentage_applying_to_victorian_uni,crow_distance_to_cbd,distance_to_cbd,duration_to_cbd,total_area,developed_area,built_percentage,urban_percentage,disturbed_percentage,water_percentage,developed_percentage,nature_percentage,mean_economic: top 5% suburb,mean_economic: top 1% suburb,mean_economic: gini coefficient,mean_economic: trimmed mean quarterly,mean_economic: variable interest rate,mean_economic: gdp quarterly,mean_economic: gdp per capita quarterly,mean_population: 0-4,mean_population: 5-9,mean_population: 10-14,mean_population: 15-19,mean_population: 20-24,mean_population: 25-29,mean_population: 30-34,mean_population: 35-39,mean_population: 40-44,mean_population: 45-49,mean_population: 50-54,mean_population: 55-59,mean_population: 60-64,mean_population: 65-69,mean_population: 70-74,mean_population: 75-79,mean_population: 80-84,mean_population: 85 and over,mean_population: total persons,mean_population: growth,mean_birth: brazil,mean_birth: wales,mean_birth: papua new guinea,mean_birth: australia,mean_birth: malaysia,mean_birth: country of birth not stated,mean_birth: indonesia,mean_birth: egypt,mean_birth: afghanistan age,mean_birth: thailand,mean_birth: ireland,mean_birth: hong kong sar of china,mean_birth: nepal,mean_birth: north macedonia,mean_birth: england,mean_birth: chile,mean_birth: fiji,mean_birth: taiwan,mean_birth: japan,mean_birth: malta,mean_birth: scotland,mean_birth: china excludes sars and taiwan,mean_birth: poland,mean_birth: singapore,mean_birth: south africa,mean_birth: greece,mean_birth: pakistan,mean_birth: iraq,mean_birth: mauritius,mean_birth: samoa,mean_birth: sri lanka,mean_birth: india,mean_birth: philippines,mean_birth: vietnam,mean_birth: new zealand,mean_birth: iran,mean_birth: lebanon,mean_birth: france,mean_birth: total,mean_birth: netherlands,mean_birth: myanmar,mean_birth: united states of america,mean_birth: bosnia and herzegovina,mean_birth: zimbabwe,mean_birth: bangladesh,mean_birth: turkey,mean_birth: canada,mean_birth: korea republic of south,mean_birth: cambodia,mean_birth: born elsewhere,mean_birth: italy,mean_birth: croatia,mean_birth: germany,mean_studying: primary other,mean_studying: tertiary FT 14-25,mean_studying: tertiary FT 25+,mean_studying: primary total,mean_studying: tertiary PT 14-25,mean_studying: tertiary PT 25+,mean_studying: tafe total,m

In [63]:
(finished_df_full.count(), len(finished_df_full.columns))

(3936, 289)

In [64]:
finished_df.write.csv("../../data/curated/finished_df.csv", header=True, mode="overwrite")
finished_df_full.write.csv("../../data/curated/finished_df_full.csv", header=True, mode="overwrite")